In [ ]:
import os
import sqlite3
import getpass as gp
import hashlib
import requests
from bs4 import BeautifulSoup


path1 = 'python_os_memory.db'
conn = sqlite3.connect(path1)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS useraccounts (
    name TEXT NOT NULL,
    pw TEXT NOT NULL
)
""")

print("Welcome to PyoNB OS!")

def check_for_accounts(db_file):
    """Checks if there are any accounts."""
    try:
        with sqlite3.connect(db_file) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM useraccounts")
            return cursor.fetchone()[0] > 0
    except sqlite3.OperationalError as e:
        print(f"Error: {e}")
        return False

def update_user_credentials(db_file, username, password):
    """Updates or inserts user credentials."""
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    try:
        with sqlite3.connect(db_file) as conn:
            cursor = conn.cursor()
            cursor.execute("UPDATE useraccounts SET pw = ? WHERE name = ?", (hashed_password, username))
            if cursor.rowcount == 0:
                cursor.execute("INSERT INTO useraccounts (name, pw) VALUES (?, ?)", (username, hashed_password))
                print(f"New account created for '{username}'.")
            else:
                print("Credentials updated successfully.")
            conn.commit()
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

def extract_text_from_url(url):
    """Fetches and extracts text from a URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        text = " ".join(soup.get_text(" ", strip=True).split())
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

def login(db_file):
    """Handles the login process."""
    while True:
        username = input("Enter your name (or 'back' to go back): ")
        if username.lower() == 'back':
            return False

        password = gp.getpass("Password: ")
        hashed_password = hashlib.sha256(password.encode()).hexdigest()

        try:
            with sqlite3.connect(db_file) as conn:
                cursor = conn.cursor()
                cursor.execute("SELECT pw FROM useraccounts WHERE name = ?", (username,))
                result = cursor.fetchone()
                if result and hashed_password == result[0]:
                    print("Login successful!")
                    return True
                else:
                    print("Incorrect username or password.")
        except sqlite3.Error as e:
            print(f"An error occurred: {e}")



while True:  # Main OS loop
    user_input = input("1. Sign in, 2. Make new account, or 3. Exit? ")

    if user_input == "1":
        if check_for_accounts(path1):
            if login(path1):
                break  # logged in successfully
        else:
            print("No accounts detected. Please create an account.")
    elif user_input == "2":
        username = input("Please enter a name: ")
        password = gp.getpass("Please enter a password: ")
        update_user_credentials(path1, username, password)
    elif user_input == "3":
        print("Exiting PyoNB OS.")
        break
        exit()
    else:
        print("Invalid choice.")

def write_essay_to_file(filename, essay_text):
    """Writes essay text to a file, preserving indentation and newlines.

    Args:
        filename: The name of the file to create (e.g., "my_essay.txt").
        essay_text: The essay text as a string, potentially with indents and newlines.
    """
    try:
        with open(filename, 'w') as f:
            f.write(essay_text)
        print(f"Essay saved to {filename}")
    except Exception as e:  # Handle potential errors like file access issues
        print(f"An error occurred: {e}")




def interactive_essay_writer(filename):
    """Interactively takes essay input and saves it to a file."""

    lines = []
    print("Enter your essay text (type 'done' on a new line to finish):")
    while True:
        line = input()
        if line.lower().strip() == "done":
            break
        lines.append(line)

    essay_text = "\n".join(lines)  # Join lines with newlines
    write_essay_to_file(filename, essay_text)

import sqlite3
import datetime

def chat(db_file):
    """
    A simple chat messaging program interacting with an SQLite database.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Ensure the table exists.  Good practice to check for its existence before creating.
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS chats (
            chatswith TEXT NOT NULL,
            messages_with_user TEXT,
            messageswithuserid INTEGER NOT NULL,  -- Use INTEGER for numeric columns
            unread_messages INTEGER DEFAULT 0      -- Use INTEGER and DEFAULT 0 for unread count
        )
    ''')
    conn.commit()


    def display_contacts():
        cursor.execute("SELECT DISTINCT chatswith FROM chats")
        contacts = cursor.fetchall()
        if contacts:
            print("\nYour contacts:")
            for i, contact in enumerate(contacts):
                print(f"{i+1}. {contact[0]}")
        else:
            print("You have no contacts yet.")
        return contacts



    def view_messages(contact):
        cursor.execute(
            "SELECT messages_with_user, messageswithuserid, unread_messages FROM chats WHERE chatswith=?", (contact,)
        )
        messages = cursor.fetchall()

        if messages:
            print(f"\nMessages with {contact}:")
            for message_content, user_id, unread in messages:  # unpack the tuple in the loop
                unread_marker = "[Unread]" if unread else "" # Indicate unread messages
                print(f"{'You:' if user_id == my_user_id else contact + ':'} {message_content} {unread_marker}")

                #Mark message as read after displaying
                if unread:
                    cursor.execute("UPDATE chats SET unread_messages = 0 WHERE chatswith = ? AND messages_with_user = ?", (contact, message_content))
                    conn.commit()
        else:
            print(f"No messages with {contact} yet.")



    def send_message(contact):
        message = input("Enter your message: ")
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Include timestamp
        cursor.execute("INSERT INTO chats (chatswith, messages_with_user, messageswithuserid, unread_messages) VALUES (?, ?, ?, ?)", (contact, message, my_user_id, 0))
        conn.commit()
        print("Message sent!")




    def add_contact():
        new_contact = input("Enter the name of the new contact: ")
        # (Optional) You might want to add validation here (e.g., check if the contact already exists).
        # ... your validation logic ...
        cursor.execute("INSERT INTO chats (chatswith, messageswithuserid) VALUES (?, ?)", (new_contact, my_user_id))  # Add initial entry for the new contact. No message content yet
        conn.commit()
        print(f"{new_contact} added to your contacts.")



    # Main chat loop
    my_user_id = int(input("Enter your user ID: "))  # Get the user ID
    while True:
        contacts = display_contacts()
        print("\nOptions:")
        print("1. View messages with a contact")
        print("2. Send a message to a contact")
        print("3. Add a new contact")
        print("4. Exit")

        choice = input("Enter your choice: ")

        if choice == '1' and contacts:
            try:
                contact_index = int(input("Enter the contact number: ")) - 1
                selected_contact = contacts[contact_index][0]
                view_messages(selected_contact)

            except (ValueError, IndexError):
                print("Invalid contact selection.")

        elif choice == '2' and contacts:

            try:
                contact_index = int(input("Enter the contact number: ")) -1
                selected_contact = contacts[contact_index][0]
                send_message(selected_contact)
            except (ValueError, IndexError):
                print("Invalid contact Selection.")


        elif choice == '3':
           add_contact()


        elif choice == '4':
            pass
        else:
            print("Invalid choice. Please try again.")

    conn.close()


while True: # Post-login command loop
  if user_input == "3":
    print("Exited.")
    break

  user_input = input("What do you want to do? (Type 'commands' for a list, 'exit' to quit): ")

  if user_input.lower() == "commands":
        print("""
            Chat - WIP.
            Notepad - Opens a notepad...
            html browser (or) bs4 - Go to a web address...
            exit - Quits the program.
        """)
  elif user_input.lower() in ("html browser", "bs4"):
      url = input("Enter URL (must have https:// extension): ")
      extracted_text = extract_text_from_url(url)
      if extracted_text:
          print(extracted_text)
  elif user_input.lower().startswith("notepad"):
      filename = input("Enter File Name: ")
      interactive_essay_writer(filename=filename)
  elif user_input.lower() == "exit":
      print("Exiting PyoNB OS.")
      break  # Correctly indented break statement
  elif user_input.lower() == "chat":
      print("Please note that this may not work in CoLab or Jupyter Notebook.")
      chat("python_os_memory.db") # Replace "chat_database.db" with your .db file name

      break  # Correctly indented break statement
  else:
        print("Invalid command.")

